# Import & Dataset 읽어오기

In [ ]:
from datasets import concatenate_datasets, load_from_disk
from src.retriever.retrieval.sparse_retrieval import SparseRetrieval
from transformers import AutoTokenizer
from src.utils.constants import column_names

org_dataset = load_from_disk('data/train_dataset')
print(org_dataset)
full_ds = concatenate_datasets(
        [
            org_dataset[column_names.TRAIN].flatten_indices(),
            org_dataset[column_names.VALIDATION].flatten_indices(),
        ]
    )  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

# Retriever 선언 및 Retriever 진행
* 아마 초기엔 학습 진행하는데 시간 좀 걸릴예정
    * 약 15분정도 소요(승범 컴 기준)
    * 노션에 mrc/검증결과/Sparse 처리에 가장 좋은 bm25올려났으므로 그거 data 폴더 안에 넣으면 실행가능.

In [ ]:
# 위에서 선언한거 가져오기 Retriever
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", use_fast=False,)
retriever = SparseRetrieval(
    tokenize_fn=tokenizer.tokenize,
    data_path="./data/",
    context_path="wikipedia_documents.json",
    mode = "bm25",
    max_feature=1000000,
    ngram_range=(1,2),
    #tokenized_docs = tokenized_docs,
)
retriever.get_sparse_embedding()

# Top k 조절.
df = retriever.retrieve(full_ds, topk=10)

# 여기서 df가 reader가 사용할 데이터 프레임입니다.
df.head(5)

* question : 질문
* id : mrc 번호
* retrieval_context : retrieval 결과로 나온 topk를 하나의 text로 만든것
* original_context : 실제 retrieval 정답
* answers: dict 형태
    * answers_start : original_context에서 어디에서 시작하는지
    * text: 정답의 text값이 뭔지 

In [ ]:
len(df[column_names.RETRIEVAL_CONTEXT][0])

## Retriever 결과물 DatasetDict로 저장
reader는 데이터셋을 DatasetDict형태로 받으므로, df에서 변환이 필요하다

In [ ]:
from datasets import Dataset, DatasetDict

# df를 Dataset으로
retriever_dataset = Dataset.from_pandas(df)

# DatasetDict로 변환 (train과 validation을 동일 데이터로 초기화)
dataset_dict = DatasetDict({
    column_names.TRAIN: retriever_dataset,
    column_names.VALIDATION: retriever_dataset
})

# train 스플릿: 'original_context'를 'context'로 사용
dataset_dict[column_names.TRAIN] = dataset_dict[column_names.TRAIN]\
    .remove_columns([column_names.RETRIEVAL_CONTEXT])\
    .rename_column(column_names.ORIGINAL_CONTEXT, column_names.CONTEXT)

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict[column_names.VALIDATION] = dataset_dict[column_names.VALIDATION]\
    .remove_columns([column_names.ORIGINAL_CONTEXT])\
    .rename_column(column_names.RETRIEVAL_CONTEXT, column_names.CONTEXT)

print(dataset_dict)


In [ ]:
# 별도 파일로 저장
dataset_dict.save_to_disk('outputs/bm25')

In [23]:
# csv파일을 읽어오기
from datasets import load_dataset, DatasetDict

# 스플릿을 validation으로 읽기 %csv 하나만 읽으면 'train' 스플릿으로 읽으니 주의
retriever_dataset = load_dataset('csv', data_files='data/test.csv')

# 불러온 Dataset 확인
print(retriever_dataset)

# train을 validation으로 변경
dataset_dict = DatasetDict({
    column_names.VALIDATION: retriever_dataset[column_names.TRAIN]
})

# validation 스플릿: 'retrieval_context'를 'context'로 사용
dataset_dict[column_names.VALIDATION] = dataset_dict[column_names.VALIDATION].remove_columns(
    column_names.UNNAMED
    ).rename_column(
        column_names.RETRIEVAL_CONTEXT, column_names.CONTEXT)

print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'question', 'id', 'retrieval_context'],
        num_rows: 600
    })
})
DatasetDict({
    validation: Dataset({
        features: ['question', 'id', 'context'],
        num_rows: 600
    })
})


In [24]:
import ast
from datasets import load_dataset, DatasetDict

# CSV 파일에서 데이터 로드
train_retriever_dataset = load_dataset('csv', data_files='data/train.csv')

# DatasetDict로 train과 validation을 정의
train_dataset_dict = DatasetDict({
    column_names.TRAIN: train_retriever_dataset[column_names.TRAIN]  # 'train' 스플릿으로 지정
})

# 불필요한 컬럼 제거 및 'retrieval_context'를 'context'로 변경
train_dataset_dict[column_names.TRAIN] = train_dataset_dict[column_names.TRAIN]\
    .remove_columns(column_names.REMOVE_COLUMNS_FROM_RETRIEVER)\
    .rename_column(column_names.RETRIEVAL_CONTEXT, column_names.CONTEXT)

# 'answers' 필드를 파싱하여 딕셔너리로 변환하는 함수
def process_answers(example):
    # 'answers' 필드가 문자열로 저장된 경우 이를 딕셔너리로 변환
    if isinstance(example[column_names.ANSWER], str):
        example[column_names.ANSWER] = ast.literal_eval(example[column_names.ANSWER])
    return example

# 'answers' 필드를 처리하여 파싱
train_dataset_dict[column_names.TRAIN] = train_dataset_dict[column_names.TRAIN]\
    .map(process_answers)

# 'train' 데이터셋의 첫 번째 예시 출력
print(train_dataset_dict[column_names.TRAIN][0])


{'question': '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?', 'id': 'mrc-1-000067', 'context': '[\'미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\\\\n\\\\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\\\\n\\\\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05\', \'국회에 관해 규정하는 헌법 제4장의 첫 조문이다.\\n\\n본

# Reader

## do train

In [26]:
from transformers import HfArgumentParser, TrainingArguments
from datasets import DatasetDict
from src.reader.model.reader import Reader
from src.utils.arguments import DataTrainingArguments, ModelArguments

# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/models/train_dataset')

# 학습 관련 설정
training_args.do_train = True
training_args.do_eval = False
training_args.do_predict = False

# 학습 하이퍼파라미터 설정
model_args.model_name_or_path = 'klue/bert-base'  # 학습에 사용할 모델
training_args.learning_rate = 5e-5  # 학습률
training_args.num_train_epochs = 1  # epoch 수
training_args.per_device_train_batch_size = 16  # 학습 배치 사이즈

# 데이터셋 로드 (BM25 기반으로 생성된 데이터셋)
dataset = train_dataset_dict

# Reader 모델 학습 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()

Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Pre-processing...
preprocessed Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 99096
})


Step,Training Loss


KeyboardInterrupt: 

## do eval

In [ ]:
# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/train_dataset')

# 평가 관련 설정
training_args.do_train = False
training_args.do_eval = True
training_args.do_predict = False

# 학습된 모델 경로 설정
model_args.model_name_or_path = 'outputs/models/train_dataset'

# 평가 데이터셋 로드 (BM25 기반으로 생성된 데이터셋)
dataset = DatasetDict.load_from_disk('outputs/bm25')

# Reader 모델 평가 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()

## do predict

In [ ]:
# Argument 설정
model_args = ModelArguments()
data_args = DataTrainingArguments()
training_args = TrainingArguments(output_dir='outputs/test_dataset')

# 추론 관련 설정
training_args.do_train = False
training_args.do_eval = False
training_args.do_predict = True

# 학습된 모델 경로 및 테스트 데이터셋 경로 설정
model_args.model_name_or_path = 'outputs/models/train_dataset'
data_args.dataset_name = 'data/test_dataset'  # 추론에 사용할 테스트 데이터셋 경로

# 테스트 데이터셋 로드
dataset = dataset_dict

# Reader 모델 추론 실행
reader_model = Reader(model_args, data_args, training_args, dataset)
reader_model.run()

In [ ]:
import json

with open('outputs/test_dataset/predictions.json', 'r', encoding='utf-8') as f:
    input_json = json.load(f)

# 변환된 데이터 저장을 위한 빈 딕셔너리
output_dict = {}

# 입력 JSON을 순회하면서 변환
for item in input_json:
    output_dict[item[column_names.ID]] = item[column_names.PREDICTION_TEST]

# 결과 출력
print(output_dict)

with open('outputs/test_dataset/converted_data.json', 'w', encoding='utf-8') as f:
    json.dump(output_dict, f, ensure_ascii=False, indent=4)